In [1]:
import pandas as pd
import numpy as np 
from autorank import autorank, create_report, plot_stats
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import sklearn.tree as tree
import os
from sklearn.metrics import accuracy_score
from scipy.stats import shapiro, wilcoxon
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm 
#euclidian distance
from scipy.spatial import distance
#warnings
import warnings
warnings.filterwarnings("ignore")
#import datasetmanager
from DatasetManager import DatasetManager
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#read data and drop irrelevant columns
df = pd.read_csv('results.csv', index_col=0)

In [2]:
dataset_ref_to_datasets = {
    "bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(1,6)],
    "sepsis_cases": ["sepsis_cases_1","sepsis_cases_2", "sepsis_cases_4"],
    "production": ["production"],

}
datasets = []
for k, v in dataset_ref_to_datasets.items():
    datasets.extend(v)

#classifiers dictionary
classifier_ref_to_classifiers = {
    "LRmodels": ["LR", "LLM", "GLRM"],
    "MLmodels": [ "XGB","RF"],
    "DLmodels": ['LSTM','CNN'],
}
classifiers = []
for k, v in classifier_ref_to_classifiers.items():
    classifiers.extend(v)

In [3]:
len(df)

91

In [4]:
len(classifiers)*len(datasets)

91

In [5]:
df[df.dataset_name.isin(['bpic2015_1_f2','bpic2015_2_f2','bpic2015_3_f2','bpic2015_4_f2','bpic2015_5_f2'])]

,dataset_name,cls_encoding,classifier,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,perc_event_total,perc_case_total,perc_control_total,FC event,FC_case,FC control,monotonicity,LOD
28,bpic2015_1_f2,agg,LR,93.42,88,35,266,0.0,3.0,12.0,0.00,8.57,4.51,0.00,20.00,80.00,0.00,1.46,95.84,0.47,2.83
29,bpic2015_1_f2,agg,LLM,93.67,88,35,266,49.5,24.5,149.5,55.68,70.00,56.20,22.15,10.96,66.89,0.80,4.37,98.75,0.37,3.74
30,bpic2015_1_f2,agg,GLRM,89.21,88,35,266,0.0,2.0,4.0,0.00,5.71,1.50,0.00,33.33,66.67,0.00,26.68,94.81,0.91,1.41
31,bpic2015_1_f2,agg,XGB,87.96,88,35,266,64.0,21.0,50.0,72.73,60.00,18.80,47.41,15.56,37.04,18.27,5.70,91.78,0.45,1.41
32,bpic2015_1_f2,agg,RF,92.16,88,35,266,86.0,34.0,222.0,97.73,97.14,83.46,25.15,9.94,64.91,6.34,2.13,90.32,0.34,4.24
33,bpic2015_1_f2,embeddings,LSTM,90.29,11,17,1,11.0,1.0,1.0,100.00,5.88,100.00,84.62,7.69,7.69,0.03,0.01,0.02,0.40,1.41
34,bpic2015_1_f2,embeddings,CNN,90.66,11,17,1,11.0,1.0,1.0,100.00,5.88,100.00,84.62,7.69,7.69,0.03,0.01,0.03,-0.07,3.61
35,bpic2015_2_f2,agg,LR,94.74,68,23,300,64.0,23.0,144.0,94.12,100.00,48.00,27.71,9.96,62.34,2.11,1.00,0.05,-0.20,7.87
36,bpic2015_2_f2,agg,LLM,94.99,68,23,300,47.5,20.0,178.0,69.12,86.96,59.33,19.35,8.15,72.51,6.82,0.00,93.18,0.35,6.16
37,bpic2015_2_f2,agg,GLRM,87.18,68,23,300,4.0,1.0,7.0,5.88,4.35,2.33,33.33,8.33,58.33,88.31,84.28,93.69,0.92,1.41


In [21]:
df.sort_values(by='monotonicity', ascending=False)

,dataset_name,cls_encoding,classifier,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,perc_event_total,perc_case_total,perc_control_total,FC event,FC_case,FC control,monotonicity,LOD
2,bpic2011_f1,agg,GLRM,91.98,136,730,175,3.0,5.0,1.0,2.21,0.68,0.57,33.33,55.56,11.11,40.16,12.16,64.54,1.00,0.00
16,bpic2011_f3,agg,GLRM,98.01,132,670,166,1.0,6.0,5.0,0.76,0.90,3.01,8.33,50.00,41.67,1.37,65.19,37.80,1.00,1.41
9,bpic2011_f2,agg,GLRM,97.40,146,760,215,2.0,8.0,2.0,1.37,1.05,0.93,16.67,66.67,16.67,4.21,27.94,40.50,0.96,0.00
37,bpic2015_2_f2,agg,GLRM,87.18,68,23,300,4.0,1.0,7.0,5.88,4.35,2.33,33.33,8.33,58.33,88.31,84.28,93.69,0.92,1.41
30,bpic2015_1_f2,agg,GLRM,89.21,88,35,266,0.0,2.0,4.0,0.00,5.71,1.50,0.00,33.33,66.67,0.00,26.68,94.81,0.91,1.41
45,bpic2015_3_f2,agg,XGB,93.71,87,34,314,62.0,20.0,44.0,71.26,58.82,14.01,49.21,15.87,34.92,22.44,4.13,85.59,0.81,2.83
51,bpic2015_4_f2,agg,GLRM,91.33,67,23,261,1.0,1.0,5.0,1.49,4.35,1.92,14.29,14.29,71.43,0.00,0.00,97.23,0.78,2.45
0,bpic2011_f1,agg,LR,95.88,136,730,175,43.0,377.0,59.0,31.62,51.64,33.71,8.98,78.71,12.32,48.38,0.02,0.00,0.78,12.08
23,bpic2011_f4,agg,GLRM,80.63,143,759,204,0.0,11.0,2.0,0.00,1.45,0.98,0.00,84.62,15.38,0.00,84.21,6.29,0.72,0.00
58,bpic2015_5_f2,agg,GLRM,90.49,78,25,246,0.0,0.0,8.0,0.00,0.00,3.25,0.00,0.00,100.00,0.00,0.00,95.94,0.71,1.41


## filter

In [22]:
underperforming = list(df.groupby('dataset_name')['AUC'].mean()[df.groupby('dataset_name')['AUC'].mean()<50].index)
print(underperforming)

[]


In [23]:
# Filter out the low performing datasets (AUC<0.5)
df = df[~df.dataset_name.isin(underperforming)]

In [24]:
#not representative for XAI purpose
df = df[~df.dataset_name.isin(list(set(df.groupby('dataset_name')['AUC'].max()[df.groupby('dataset_name')['AUC'].max()<80].index)))]

In [25]:
dataset_cols = ['classifier','cls_encoding','AUC','#event columns','#case columns','#control columns','pars. event','pars. case','pars. control','perc_pars_event','perc_pars_case','perc_pars_control','FC event','FC_case','FC control','monotonicity','LOD']

In [26]:
set(df['dataset_name'])

{'bpic2011_f1',
 'bpic2011_f2',
 'bpic2011_f3',
 'bpic2011_f4',
 'bpic2015_1_f2',
 'bpic2015_2_f2',
 'bpic2015_3_f2',
 'bpic2015_4_f2',
 'bpic2015_5_f2',
 'sepsis_cases_2'}

In [36]:
bpic2015_1 = ['bpic2015_1_f2']
bpic2015_1 = df[df.dataset_name.isin(bpic2015_1)].sort_values('AUC')
bpic2015_1 = bpic2015_1[dataset_cols]
bpic2015_1 = bpic2015_1.sort_values(by='AUC')

In [37]:
bpic2015_1

,classifier,cls_encoding,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,FC event,FC_case,FC control,monotonicity,LOD
31,XGB,agg,87.96,88,35,266,64.0,21.0,50.0,72.73,60.00,18.80,18.27,5.70,91.78,0.45,1.41
30,GLRM,agg,89.21,88,35,266,0.0,2.0,4.0,0.00,5.71,1.50,0.00,26.68,94.81,0.91,1.41
33,LSTM,embeddings,90.29,11,17,1,11.0,1.0,1.0,100.00,5.88,100.00,0.03,0.01,0.02,0.40,1.41
34,CNN,embeddings,90.66,11,17,1,11.0,1.0,1.0,100.00,5.88,100.00,0.03,0.01,0.03,-0.07,3.61
32,RF,agg,92.16,88,35,266,86.0,34.0,222.0,97.73,97.14,83.46,6.34,2.13,90.32,0.34,4.24
28,LR,agg,93.42,88,35,266,0.0,3.0,12.0,0.00,8.57,4.51,0.00,1.46,95.84,0.47,2.83
29,LLM,agg,93.67,88,35,266,49.5,24.5,149.5,55.68,70.00,56.20,0.80,4.37,98.75,0.37,3.74


In [38]:
bpic2011_2 = ['bpic2011_f2']
bpic2011_2 = df[df.dataset_name.isin(bpic2011_2)].sort_values('AUC')
bpic2011_2 = bpic2011_2[dataset_cols]
bpic2011_2 = bpic2011_2.sort_values(by='AUC')

In [39]:
bpic2011_2

,classifier,cls_encoding,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,FC event,FC_case,FC control,monotonicity,LOD
13,CNN,embeddings,80.67,13,6,1,13.0,6.0,1.0,100.00,100.00,100.00,0.02,0.02,0.02,0.43,2.24
12,LSTM,embeddings,86.55,13,6,1,13.0,6.0,1.0,100.00,100.00,100.00,0.02,0.03,0.03,0.35,2.24
7,LR,agg,96.56,146,760,215,84.0,495.0,115.0,57.53,65.13,53.49,0.12,0.00,0.00,0.01,2.45
9,GLRM,agg,97.40,146,760,215,2.0,8.0,2.0,1.37,1.05,0.93,4.21,27.94,40.50,0.96,0.00
8,LLM,agg,98.18,146,760,215,122.5,532.0,182.0,83.56,70.00,84.65,28.69,0.00,79.81,0.02,8.30
10,XGB,agg,98.21,146,760,215,69.0,99.0,60.0,47.26,13.03,27.91,62.32,11.41,26.61,0.39,4.24
11,RF,agg,98.49,146,760,215,134.0,760.0,207.0,91.78,100.00,96.28,27.78,28.12,24.53,0.11,9.90


In [40]:
bpic2015_1

,classifier,cls_encoding,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,FC event,FC_case,FC control,monotonicity,LOD
31,XGB,agg,87.96,88,35,266,64.0,21.0,50.0,72.73,60.00,18.80,18.27,5.70,91.78,0.45,1.41
30,GLRM,agg,89.21,88,35,266,0.0,2.0,4.0,0.00,5.71,1.50,0.00,26.68,94.81,0.91,1.41
33,LSTM,embeddings,90.29,11,17,1,11.0,1.0,1.0,100.00,5.88,100.00,0.03,0.01,0.02,0.40,1.41
34,CNN,embeddings,90.66,11,17,1,11.0,1.0,1.0,100.00,5.88,100.00,0.03,0.01,0.03,-0.07,3.61
32,RF,agg,92.16,88,35,266,86.0,34.0,222.0,97.73,97.14,83.46,6.34,2.13,90.32,0.34,4.24
28,LR,agg,93.42,88,35,266,0.0,3.0,12.0,0.00,8.57,4.51,0.00,1.46,95.84,0.47,2.83
29,LLM,agg,93.67,88,35,266,49.5,24.5,149.5,55.68,70.00,56.20,0.80,4.37,98.75,0.37,3.74


In [41]:
frames = [bpic2015_1]

In [42]:
result = pd.concat(frames)

In [43]:
result

,classifier,cls_encoding,AUC,#event columns,#case columns,#control columns,pars. event,pars. case,pars. control,perc_pars_event,perc_pars_case,perc_pars_control,FC event,FC_case,FC control,monotonicity,LOD
31,XGB,agg,87.96,88,35,266,64.0,21.0,50.0,72.73,60.00,18.80,18.27,5.70,91.78,0.45,1.41
30,GLRM,agg,89.21,88,35,266,0.0,2.0,4.0,0.00,5.71,1.50,0.00,26.68,94.81,0.91,1.41
33,LSTM,embeddings,90.29,11,17,1,11.0,1.0,1.0,100.00,5.88,100.00,0.03,0.01,0.02,0.40,1.41
34,CNN,embeddings,90.66,11,17,1,11.0,1.0,1.0,100.00,5.88,100.00,0.03,0.01,0.03,-0.07,3.61
32,RF,agg,92.16,88,35,266,86.0,34.0,222.0,97.73,97.14,83.46,6.34,2.13,90.32,0.34,4.24
28,LR,agg,93.42,88,35,266,0.0,3.0,12.0,0.00,8.57,4.51,0.00,1.46,95.84,0.47,2.83
29,LLM,agg,93.67,88,35,266,49.5,24.5,149.5,55.68,70.00,56.20,0.80,4.37,98.75,0.37,3.74


In [44]:
result.to_csv('evaluation/BPIC2015(1).csv')